In [ ]:
openai_api_key = ""
import time
import matplotlib.pyplot as plt
import numpy as np
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)

In [ ]:
BASE_MODEL = "gpt-4-turbo"  # Update to GPT-4 model

def LLM(messages, model=BASE_MODEL, stop=None, max_tokens=256, temperature=0.0):  #Defualt = 0.9
    responses = client.chat.completions.create(
        model=model, messages=messages, max_tokens=max_tokens, 
        temperature=temperature, stop=stop
    )
    text = responses.choices[0].message.content
    return text

# Test LLM with GPT-4
LLM([{"role": "user", "content": "hello world!"}])

In [ ]:
PROMPT_dis_delta = [
    {
        "role": "user",
        "content": "I need your assistance in generating the next incremental step for a path from the yellow box to the red box, passing through the green box, while avoiding the blue area obstacle.\n\n"
                   "Environment:\n"
                   "- Object 1: [red box, center position: [-1.5, 1.2]]\n"
                   "- Object 2: [yellow box, center position: [1, -0.5]]\n"
                   "- Object 3: [green box, center position: [-1, 0]]\n"
                   "- Obstacle (blue area): Square with corners at [[-0.5, 0.0], [0.5, 0.0], [-0.5, -1.0], [0.5, -1.0]], which must be completely avoided.\n\n"
                   "Instructions:\n"
                   "- Only output `delta_X` and `delta_Y` values for the next point in the path.\n"
                   "- Ensure that each step avoids the blue area and follows the directional restrictions given by the blue area’s corners.\n"
                   "- The `delta_X` and `delta_Y` values should guide the path smoothly, avoiding abrupt changes.\n\n"

                   "Provide the next step in this format:\n"
                   "{\n"
                   "    \"delta_X\": value,\n"
                   "    \"delta_Y\": value\n"
                   "}\n\n"

                   "Note: Only output one set of `delta_X` and `delta_Y` for each response."
                   "Dont' repeat the planning problem and context of the prompt"
    }
]

# Use the prompt to request each incremental step.
delta_step = LLM(PROMPT_dis_delta)
print(delta_step)

In [ ]:
#V4 --better than >> V5
import json
import re
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Polygon

# Initial and goal positions
current_x, current_y = 1, -0.5  # Initial position (I)
goal_x, goal_y = -1.5, 1.2      # Goal position (G)

# Define general quadrilateral obstacles
obstacles = [
    Polygon([[0.0, -.15], [0.5, 0.3], [0.3, 0.8], [-0.2, 0.4]]),  # Quadrilateral 1
    Polygon([[-1.0, 0.0], [-0.6, 0.4], [-1.2, 0.7], [-1.4, 0.3]])  # Quadrilateral 2
]

# Path storage
path_coordinates = [(current_x, current_y)]
iteration_count = 0
max_iterations = 100
m, q = 5, 2  # Predict `m` positions, execute `q` per iteration

# Helper function to check if a line segment is disjoint from all obstacles
def is_line_disjoint(line_start, line_end, obstacles):
    line = LineString([line_start, line_end])
    for obstacle in obstacles:
        if line.intersects(obstacle):
            return False
    return True

# Helper function to generate the LLM prompt
def generate_prompt(current_x, current_y, goal_x, goal_y, obstacles, replanning_mode=False):
    obstacles_description = "\n".join(
        f"- Obstacle {i + 1}: Vertices {list(obstacle.exterior.coords)[:-1]}"
        for i, obstacle in enumerate(obstacles)
    )
    exploration_hint = (
        "\n\nReplanning Exploration Hints:\n"
        "- Consider curved segments or deviations from the straight path.\n"
        "- Explore alternative directions away from obstacles.\n"
        "- Use step-by-step reasoning to identify feasible exploratory paths."
        if replanning_mode else ""
    )

    return [
        {
            "role": "user",
            "content": (
                f"You are a reasoning agent tasked with planning a correct path for a robot.\n\n"
                f"Environment:\n"
                f"- Initial position (current): [{current_x}, {current_y}]\n"
                f"- Goal position: [{goal_x}, {goal_y}]\n"
                f"- Obstacles:\n{obstacles_description}\n\n"
                "Instructions:\n"
                "1. Plan the next 5 positions (m = 5) toward the goal.\n"
                "2. Ensure the straight line between each consecutive position is disjoint from all obstacles.\n"
                "3. If replanning, include exploration by considering curves or alternative directions.\n"
                "4. Provide step-by-step reasoning in your planning to ensure constraints are met."
                "5. Coordinates (Xi, Yi) for each position must have a precision greater than 0.1 (e.g., 0.05 or 0.01)."
                "6. If planning stuks in feedback loop, use smaller and finer points"

                f"{exploration_hint}\n\n"
                "Output format:\n"
                "{\n"
                "  \"Positions\": [\n"
                "    [X1, Y1],\n"
                "    [X2, Y2],\n"
                "    [X3, Y3],\n"
                "    [X4, Y4],\n"
                "    [X5, Y5]\n"
                "  ]\n"
                "}\n\n"
                "Important:\n"
                "- Only provide the JSON object with the list of positions.\n"
                "- Do not include explanations, repeated context, or extra formatting."
            )
        }
    ]

# Loop until the goal is reached or maximum iterations are hit
replanning_mode = False
while (abs(current_x - goal_x) > 0.1 or abs(current_y - goal_y) > 0.1) and iteration_count < max_iterations:
    iteration_count += 1

    # Generate the GPT prompt
    prompt = generate_prompt(current_x, current_y, goal_x, goal_y, obstacles, replanning_mode)

    # Request positions from GPT
    positions_text = LLM(prompt)
    json_match = re.search(r"\{[\s\S]*\}", positions_text)
    if json_match:
        positions_text = json_match.group(0)

    try:
        positions = json.loads(positions_text)["Positions"]

        # Print the received positions
        print(f"\nIteration {iteration_count} ({'Replanning' if replanning_mode else 'Planning'} Mode): Predicted Positions:")
        for idx, (pos_x, pos_y) in enumerate(positions):
            print(f"  Position {idx + 1}: [{pos_x}, {pos_y}]")

        # Validate and execute the first q valid steps
        executed_positions = []
        for pos_x, pos_y in positions:
            if is_line_disjoint((current_x, current_y), (pos_x, pos_y), obstacles):
                executed_positions.append((pos_x, pos_y))

        # If no valid positions are found, enter replanning mode
        if not executed_positions:
            print(f"No valid positions found in iteration {iteration_count}. Entering replanning mode...")
            replanning_mode = True
            continue  # Retry planning with exploration hints

        # Apply only the first q valid positions
        print(f"  Executing {min(q, len(executed_positions))} valid positions:")
        for i in range(min(q, len(executed_positions))):
            current_x, current_y = executed_positions[i]
            path_coordinates.append((current_x, current_y))  # Add to path
            print(f"    Applied Position {i + 1}: [{current_x}, {current_y}]")

        # Exit replanning mode if successful
        replanning_mode = False

    except (json.JSONDecodeError, KeyError):
        print("Error parsing positions from LLM response.")
        continue

# Plot the entire path
path_array = np.array(path_coordinates)
plt.plot(path_array[:, 0], path_array[:, 1], marker="o", linestyle='-', color="tab:blue", linewidth=2)
plt.scatter(path_array[0, 0], path_array[0, 1], color="green", s=100, label="Start")
plt.scatter(goal_x, goal_y, color="red", s=100, label="Goal")

# Plot quadrilateral obstacles
for i, obstacle in enumerate(obstacles):
    x, y = obstacle.exterior.xy
    plt.fill(x, y, alpha=0.5, label=f"Obstacle {i + 1}")

plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.title("Path Planning with Replanning and Exploration")
plt.legend()
plt.grid()
plt.show()
